In [1]:
import nltk
import pandas as pd
import numpy as np
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy
import re, string

In [2]:
reviews = pd.read_excel('../data/airbnb_text_reviews.xlsx')

In [3]:
reviews.sample(1)

,listing_id,id,date,reviewer_id,reviewer_name,comments
112515,1370405,21147209,2014-10-12,21057777,Andre,Excellent value for money! Christa and Zach we...


In [4]:
reviews.sample(3)

,listing_id,id,date,reviewer_id,reviewer_name,comments
359415,11775483,505044601,2019-08-08,2743522,Alexandra,Anthony was very responsive and helpful to me ...
341417,10482409,183225312,2017-08-16,138653197,Elijah,Perfect place to stay on a budget in the middl...
1027629,54221901,582576044744759040,2022-03-14,167799886,James,"Spacious, clean, great location."


In [5]:
reviews['comments'] = reviews['comments'].astype('string')

In [6]:
reviews['comments'].describe()

count         1048358
unique        1000410
top       Great place
freq             1366
Name: comments, dtype: object

## removing rows with only punctuation

In [7]:
pattern = re.compile("[\d{}]+$".format(re.escape(string.punctuation)))

In [8]:
reviews.loc[(reviews['comments'] == '.') | (reviews['comments'] == '!') | (reviews['comments'] == '?') ]

,listing_id,id,date,reviewer_id,reviewer_name,comments
609,5178,458595941119654016,2021-09-24,409836786,Elya,.
8075,27644,503596934129974976,2021-11-25,38781491,Jay,.
9103,31994,180524495,2017-08-10,98704132,Martin (Aka Max),.
9106,31994,194910000,2017-09-17,99756545,Vardhan,.
9159,31994,366953207,2019-01-03,213464080,Glenda,.
...,...,...,...,...,...,...
1045288,593313785791808000,645676525698039040,2022-06-09,461019684,Devon,.
1046154,596511810816093056,618129884161660032,2022-05-02,455819359,Tatiana,.
1046404,597180195652908032,610077003524514944,2022-04-21,421646015,Guillermo,.
1047280,599993984917025024,623911135936929024,2022-05-10,46317236,Alexandre,.


In [ ]:
## 

## Lematizing and POS with SpaCy

In [26]:
reviews['comments'].dropna(inplace = True)

In [27]:
review_list = reviews['comments'][:10000].to_list()
review_list[0]

'Great host '

In [28]:
for index, string in enumerate(review_list):
  if type(string) != type('example'):
    review_list[index] = ""

In [34]:
review_string = '@'.join(review_list)
review_string[:100]

'Great host @Nice room for the price. Great neighborhood. John was very accommodating. Bottles of wat'

In [35]:
split_text = [review_string[i:i+800000] for i in range(0, len(review_string), 800000)]

In [36]:
# Initialize spacy 'en_core_web_sm' model, keeping only tagger component needed for lemmatization

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

Doc = nlp('') # define an empty doc object

In [37]:
# create a doc object by iterating through our corpus
docs = []
for piece in split_text:
    doc = nlp(piece)
    docs.append(doc)

merged_doc = Doc.from_docs(docs)

In [39]:
# Extract the lemma for each token and join
lemmatized_corpus = " ".join([token.lemma_.lower() for token in merged_doc if not (token.is_punct or pattern.match(token.text))])

lemmatized_corpus_list = lemmatized_corpus.split(sep = "@")

In [40]:
len(lemmatized_corpus)

lemmatized_corpus_list

['great host ',
 'nice room for the price great neighborhood john be very accommodating bottle of water in the room be a nice touch and very much appreciated.',
 'very nice apt   new remodeled.',
 "great place to stay for a while john be a great host and a great man the house be very homy you 'll feel right at home    .",
 "i really enjoy my time here in deep south brooklyn and john be an excellent host very attentive to any concern that i have and a very welcome presence when he be around br/><br/>the room itself be a great space with a super comfy bed and lovely traditional ceiling fan the street be pleasant tree line and quiet and the townhouse be locate a short convenient walk from the f train which have be perfect for get to my office here.<br/><br/>it 's easy to see why john 's a super host and i look forward to stay with he again!",
 'john be a great host he be very responsive before and during our visit he really want to make sure we have a good stay it be our 1st airbnb experi

## Time to clean our text

In [ ]:
reviews_1000 = reviews.head(1000)

reviews_1000['cleaned_tokens'] = lemmatized_corpus_list